# GEMS (Convex) Demo

A brief demonstration of GEMS for convex models, applied to MNIST (5 nodes) 

In [1]:
# Import relevant libraries 
import os

from utils.data_utils import *
from utils.model_utils import *
from utils.logit_ellipsoid import *

%matplotlib notebook
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
# Set experiment configurations
exp = {
    "dataset": 'mnist',                   # Specify dataset
    "num_nodes": 5,                       # Specify number of nodes
    "model": "logit",                     # Specify model type 
    "local_epochs": 40,                   # Specify number of local training epochs
    "optimizer": "adam",                  # Specify optimizer 
    "loss": 'categorical_crossentropy',   # Specify loss function
    'final_epsilon': 0.70,                # Specify epsilon (accuracy) for final layer 
    'max_radius': 4000.0,                 # Specify maximum radius to use for model 
    'num_samples': 500,                   # Specify number of models to sample from epsilloid surface
    'fisher_count': 200,                  # Specify number of samples to use for fisher information matrix calculation
    'ellipse': True,                      # Specify good-enough model space geometry
    'ellipse_type': 'fisher',             # Specify radius expansion strategy 
    'holdout_trials': 5,                  # Number of trials to evaluate results over 
    'holdout_sizes': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000] # number of hold out samples to try tuning on
}

In [3]:
# Load data 
data_dir = os.path.join("../datasets", "%s_%d" % (exp['dataset'], exp['num_nodes']))
Xtr, ytr, Xval, yval, Xts, yts = load_data(data_dir)
Xts_flat, yts_flat = flatten(Xts, yts)
Xtr_flat, ytr_flat = flatten(Xtr, ytr)

In [4]:
# Train local models 
# Run local
all_results = {}
models = []
for t in range(exp['num_nodes']):
    print("Training local model on node-%d..." % t)
    model, results = train_local_model(Xtr, ytr, Xts, yts, t, exp)
    models.append(model)

    all_results[t] = results

Training local model on node-0...
Training local model on node-1...
Training local model on node-2...
Training local model on node-3...
Training local model on node-4...


In [6]:
# Compute the GEMS model 
gems_model, results = run_logit_ellipsoid(exp, Xtr, ytr, Xval, yval, Xts, yts, models)

Computing fisher information matrix for node-0
Computing fisher information matrix for node-1
Computing fisher information matrix for node-2
Computing fisher information matrix for node-3
Computing fisher information matrix for node-4
Computing good-enough model space for node-0
Node-0. Radius: 43.609619
Computing good-enough model space for node-1
Node-1. Radius: 20.660400
Computing good-enough model space for node-2
Node-2. Radius: 35.430908
Computing good-enough model space for node-3
Node-3. Radius: 42.449951
Computing good-enough model space for node-4
Node-4. Radius: 25.482178
GEMS accuracy: 0.449400
Holdout Size: 100.
	Raw: 0.364640.
	Gems (Tuned): 0.735240.
	Average (Tuned): 0.749160.
	 Node-0 Loc Model (Tuned): 0.211000
	 Node-1 Loc Model (Tuned): 0.205280
	 Node-2 Loc Model (Tuned): 0.181880
	 Node-3 Loc Model (Tuned): 0.197400
	 Node-4 Loc Model (Tuned): 0.194680
Holdout Size: 200.
	Raw: 0.580000.
	Gems (Tuned): 0.817400.
	Average (Tuned): 0.819000.
	 Node-0 Loc Model (Tuned